In [1]:
import sqlite3
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

C:\Users\delta\anaconda3\envs\py3_7_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\delta\anaconda3\envs\py3_7_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\delta\anaconda3\envs\py3_7_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\delta\anaconda3\envs\py3_7_tf\lib\site-packages\tensorf

In [2]:
db_file = "teemo-tool-data_2020-07-31_01.db"

In [3]:
conn = sqlite3.connect(db_file)

In [4]:
cur = conn.cursor()

In [5]:
sql_str = """
SELECT match_json
FROM matches LIMIT 5
"""

In [6]:
cur.execute(sql_str)

In [7]:
rows = cur.fetchall()

In [8]:
json_data = rows[0][0]

In [9]:
loaded = json.loads(json_data)
loaded

{'gameId': 3511863061,
 'platformId': 'NA1',
 'gameCreation': 1595919773332,
 'gameDuration': 2206,
 'queueId': 420,
 'mapId': 11,
 'seasonId': 13,
 'gameVersion': '10.15.328.8759',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'teams': [{'teamId': 100,
   'win': 'Fail',
   'firstBlood': True,
   'firstTower': False,
   'firstInhibitor': False,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 0,
   'inhibitorKills': 0,
   'baronKills': 0,
   'dragonKills': 0,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': [{'championId': 161, 'pickTurn': 1},
    {'championId': 51, 'pickTurn': 2},
    {'championId': 157, 'pickTurn': 3},
    {'championId': 82, 'pickTurn': 4},
    {'championId': -1, 'pickTurn': 5}]},
  {'teamId': 200,
   'win': 'Win',
   'firstBlood': False,
   'firstTower': True,
   'firstInhibitor': True,
   'firstBaron': False,
   'firstDragon': True,
   'firstRiftHerald': True,
   'towerKill

In [10]:
teams = loaded["teams"]
team_results = list(map(lambda x: {"teamId": x["teamId"], "win": x["win"]}, teams))
team_results

[{'teamId': 100, 'win': 'Fail'}, {'teamId': 200, 'win': 'Win'}]

In [11]:
participants = loaded["participants"]
champions = list(map(lambda x: {"teamId": x["teamId"], "championId": x["championId"]}, participants))
champions

[{'teamId': 100, 'championId': 101},
 {'teamId': 100, 'championId': 22},
 {'teamId': 100, 'championId': 56},
 {'teamId': 100, 'championId': 876},
 {'teamId': 100, 'championId': 17},
 {'teamId': 200, 'championId': 91},
 {'teamId': 200, 'championId': 141},
 {'teamId': 200, 'championId': 86},
 {'teamId': 200, 'championId': 110},
 {'teamId': 200, 'championId': 267}]

In [12]:
def get_all_match_jsons(cur):
    all_match_json_sql = """
    SELECT match_json
    FROM matches
    """
    cur.execute(all_match_json_sql)
    all_match_json_rows = cur.fetchall()
    
    return list(map(lambda row: row[0], all_match_json_rows))

In [13]:
def get_unique_champion_id_list(cur):
    all_match_jsons = get_all_match_jsons(cur)
    all_match_jsons = map(lambda x: json.loads(x), all_match_jsons)
    all_match_participants = map(lambda x: x["participants"], all_match_jsons)
    all_match_champion_ids = map(lambda participants: [participant["championId"] for participant in participants], all_match_participants)
    flattened_champion_ids = [item for sublist in all_match_champion_ids for item in sublist]
    return list(set(flattened_champion_ids))

In [14]:
unique_champ_id_list = get_unique_champion_id_list(cur)

In [15]:
def make_team_comp_vector(team_id, match_dict, unique_champion_ids):
    champ_id_to_vec_pos_mapping = {unique_champion_ids[i]: i for i in range(len(unique_champion_ids))}
    vec = np.zeros(len(unique_champion_ids))
    for participant in match_dict["participants"]:
        if participant["teamId"] != team_id:
            continue
        
        vec_pos = champ_id_to_vec_pos_mapping[participant["championId"]]
        vec[vec_pos] = 1
    return vec

In [16]:
make_team_comp_vector(100, loaded, unique_champ_id_list)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
def make_match_vector(match_json, unique_champ_id_list):
    match_dict = json.loads(match_json)
    team100_vector = make_team_comp_vector(100, match_dict, unique_champ_id_list)
    team200_vector = make_team_comp_vector(200, match_dict, unique_champ_id_list)
    return np.concatenate((team100_vector, team200_vector))

In [18]:
make_match_vector(json_data, unique_champ_id_list)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [19]:
def make_match_result(match_json):
    # returns 1 if team100 won
    # otherwise 0
    match_dict = json.loads(match_json)
    team_dict = match_dict["teams"][0]
    team_id = team_dict["teamId"]
    team_win = team_dict["win"]
    if team_id == 100:
        return 1 if team_win == "Win" else 0
    if team_id == 200:
        return 0 if team_win == "Win" else 1

In [20]:
make_match_result(json_data)

0

In [21]:
all_match_jsons = get_all_match_jsons(cur)
X = np.array([make_match_vector(match_json, unique_champ_id_list) for match_json in all_match_jsons])
y = np.array([make_match_result(match_json) for match_json in all_match_jsons])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9994)

In [23]:
len(X_train[:,0])

69240

In [24]:
len(X_train[1,:])

298

In [25]:
inputs = keras.Input(shape=(len(unique_champ_id_list)*2,))

In [26]:
x = layers.Dense(32, activation="relu")(inputs)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
output = layers.Dense(1, activation='sigmoid')(x)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
model = keras.Model(inputs=inputs, outputs=output)
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=5)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
69240/69240 [==============================] - 31s 454us/sample - loss: 0.6890 - acc: 0.5343
Epoch 2/10
69240/69240 [==============================] - 26s 374us/sample - loss: 0.6815 - acc: 0.5616
Epoch 3/10
69240/69240 [==============================] - 27s 383us/sample - loss: 0.6750 - acc: 0.5754
Epoch 4/10
69240/69240 [==============================] - 34s 491us/sample - loss: 0.6661 - acc: 0.5908
Epoch 5/10
69240/69240 [==============================] - 31s 441us/sample - loss: 0.6559 - acc: 0.6068
Epoch 6/10
69240/69240 [==============================] - 33s 471us/sample - loss: 0.6452 - acc: 0.6208
Epoch 7/10
69240/69240 [==============================] - 32s 464us/sample - loss: 0.6353 - acc: 0.6318
Epoch 8/10
69240/69240 [==============================] - 32s 459us/sample - loss: 0.6264 - acc: 0.6407
Epoch 9/10
69240/69240 [==============================] - 31s 454us/sample

[0.7474700901289333, 0.5267475]

In [33]:
# svm_class = svm.SVC(kernel="rbf")
# svm_class.fit(X_train, y_train)
# y_test_pred = svm_class.predict(X_test)

In [34]:
# print(accuracy_score(y_test, y_test_pred))

0.5453495089543616


In [ ]:
# svm_class.score(X_train, y_train)

In [ ]:
# svm_class.score(X_test, y_test)

In [28]:
logreg_class = LogisticRegression(n_jobs=10)
logreg_class.fit(X_train, y_train)
y_test_pred_logreg = logreg_class.predict(X_test)

In [29]:
print(accuracy_score(y_test, y_test_pred_logreg))

0.5501444251877528


In [30]:
logreg_class.score(X_train, y_train)

0.5650057770075101

In [31]:
logreg_class.score(X_test, y_test)

0.5501444251877528

In [32]:
inputs2 = keras.Input(shape=(len(unique_champ_id_list)*2,))
x2 = layers.Dense(32, activation="relu")(inputs2)
x2 = layers.Dense(32, activation="relu")(x2)
x2 = layers.Dense(16, activation="relu")(x2)
x2 = layers.Dense(16, activation="relu")(x2)
output2 = layers.Dense(1, activation='sigmoid')(x2)
model2 = keras.Model(inputs=inputs2, outputs=output2)
model2.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
model2.fit(X_train, y_train, epochs=20, batch_size=8)
model2.evaluate(X_test, y_test)

Epoch 1/20
69240/69240 [==============================] - 64s 922us/sample - loss: 0.6884 - acc: 0.5380
Epoch 2/20
69240/69240 [==============================] - 109s 2ms/sample - loss: 0.6814 - acc: 0.5610
Epoch 3/20
69240/69240 [==============================] - 25s 361us/sample - loss: 0.6748 - acc: 0.5750
Epoch 4/20
69240/69240 [==============================] - 26s 376us/sample - loss: 0.6642 - acc: 0.5938
Epoch 5/20
69240/69240 [==============================] - 23s 333us/sample - loss: 0.6533 - acc: 0.6106
Epoch 6/20
69240/69240 [==============================] - 24s 348us/sample - loss: 0.6431 - acc: 0.6241
Epoch 7/20
69240/69240 [==============================] - 25s 368us/sample - loss: 0.6333 - acc: 0.6347
Epoch 8/20
69240/69240 [==============================] - 29s 418us/sample - loss: 0.6249 - acc: 0.6445
Epoch 9/20
69240/69240 [==============================] - 28s 411us/sample - loss: 0.6170 - acc: 0.6529
Epoch 10/20
69240/69240 [==============================] - 31s 44

[0.8251649287792384, 0.522877]

In [25]:
inputs3 = keras.Input(shape=(len(unique_champ_id_list)*2,))
x3 = layers.Dense(64, activation="relu")(inputs3)
x3 = layers.Dense(32, activation="relu")(x3)
x3 = layers.Dense(32, activation="relu")(x3)
x3 = layers.Dense(16, activation="relu")(x3)
output3 = layers.Dense(1, activation='sigmoid')(x3)
model3 = keras.Model(inputs=inputs3, outputs=output3)
model3.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
model3.fit(X_train, y_train, epochs=40, batch_size=15)
model3.evaluate(X_test, y_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/40
69240/69240 [==============================] - 19s 274us/sample - loss: 0.6883 - acc: 0.5371
Epoch 2/40
69240/69240 [==============================] - 18s 253us/sample - loss: 0.6799 - acc: 0.5652
Epoch 3/40
69240/69240 [==============================] - 15s 211us/sample - loss: 0.6694 - acc: 0.5854
Epoch 4/40
69240/69240 [==============================] - 18s 255us/sample - loss: 0.6507 - acc: 0.6147
Epoch 5/40
69240/69240 [==============================] - 16s 237us/sample - loss: 0.6286 - acc: 0.6418
Epoch 6/40
69240/69240 [==============================] - 18s 257us/sample - loss: 0.6063 - acc: 0.6652
Epoch 7/40
69240/69240 [==============================] - 17s 252us/sample - loss: 0.5869 - acc: 0.6803
Epoch 8/40
69240/69240 [==============================] - 16

[1.5551981527911385, 0.5183131]

In [26]:
inputs4 = keras.Input(shape=(len(unique_champ_id_list)*2,))
x4 = layers.Dense(128, activation="relu")(inputs4)
x4 = layers.Dense(64, activation="relu")(x4)
x4 = layers.Dense(32, activation="relu")(x4)
x4 = layers.Dense(32, activation="relu")(x4)
x4 = layers.Dense(16, activation="relu")(x4)
output4 = layers.Dense(1, activation='sigmoid')(x4)
model4 = keras.Model(inputs=inputs4, outputs=output4)
model4.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
model4.fit(X_train, y_train, epochs=80, batch_size=15)
model4.evaluate(X_test, y_test)

Epoch 1/80
69240/69240 [==============================] - 18s 266us/sample - loss: 0.6883 - acc: 0.5374
Epoch 2/80
69240/69240 [==============================] - 19s 279us/sample - loss: 0.6799 - acc: 0.5641
Epoch 3/80
69240/69240 [==============================] - 20s 289us/sample - loss: 0.6679 - acc: 0.5885
Epoch 4/80
69240/69240 [==============================] - 18s 258us/sample - loss: 0.6364 - acc: 0.6337
Epoch 5/80
69240/69240 [==============================] - 20s 284us/sample - loss: 0.5911 - acc: 0.6799
Epoch 6/80
69240/69240 [==============================] - 18s 266us/sample - loss: 0.5463 - acc: 0.7178
Epoch 7/80
69240/69240 [==============================] - 19s 272us/sample - loss: 0.5047 - acc: 0.7471
Epoch 8/80
69240/69240 [==============================] - 20s 289us/sample - loss: 0.4664 - acc: 0.7724
Epoch 9/80
69240/69240 [==============================] - 19s 276us/sample - loss: 0.4328 - acc: 0.7938
Epoch 10/80
69240/69240 [==============================] - 23s 3

69240/69240 [==============================] - 20s 288us/sample - loss: 0.0425 - acc: 0.9851
Epoch 80/80
17310/17310 [==============================] - 1s 56us/sample - loss: 5.5693 - acc: 0.5126


[5.569326660799746, 0.51259387]